## Skullstrip the LGG NIFTIS

For each exam, we want to find the T1, T2, T2FLAIR, and T1C and for the first go, we're just going to skull strip using the BET program from FSL. 

We want to use the following command: 

` bet <input> <output> -S`

In [1]:
import glob
import subprocess as sub 
from subprocess import * 
import os 
from pathlib import Path
import pandas as pd

We can use the output from a prior analysis to label our images: 

In [2]:
full_label_df = pd.read_csv('/working/lupolab/julia/tcia_analysis/labels_tcia_lgg.csv')

In [3]:
full_label_df.head()

,ground_truth,image,sd
0,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,305-RT__COW-39894
1,T1,/working/lupolab/julia/tcia_analysis/datasets/...,1101-T1_AX_SE-54608
2,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,404-eB1000i-30080
3,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,301-3D_MOTSA_sense-80049
4,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,403-eBo-48146


In [4]:
full_label_df.loc[0]['image'].split('/')[7]

'TCGA-CS-5396'

In [5]:
full_label_df['patient_id'] = [x.split('/')[7] for x in full_label_df.image]

In [6]:
full_label_df['exam_id'] = [x.split('/')[8] for x in full_label_df.image]

In [7]:
full_label_df

,ground_truth,image,sd,patient_id,exam_id
0,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,305-RT__COW-39894,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
1,T1,/working/lupolab/julia/tcia_analysis/datasets/...,1101-T1_AX_SE-54608,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
2,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,404-eB1000i-30080,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
3,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,301-3D_MOTSA_sense-80049,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
4,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,403-eBo-48146,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
5,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,402-dADC_MAP-53538,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
6,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,304-LT_COW-25699,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
7,T1,/working/lupolab/julia/tcia_analysis/datasets/...,501-T1W__SE-58849,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
8,T2,/working/lupolab/julia/tcia_analysis/datasets/...,701-T2WTSE-70754,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
9,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,901-FFE_FOR_BLEED-67362,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605


In [8]:
patient_list = full_label_df.patient_id.unique()

In [9]:
len(patient_list)

199

In [10]:
usable_patients = []
for patient in patient_list: 
    patient_df = full_label_df.loc[(full_label_df['patient_id'] == patient) & (full_label_df['ground_truth'] != "OTHER")]
#     print(patient_df)
    if len(patient_df.ground_truth.unique()) >= 4: 
        usable_patients.append(patient)

        

In [11]:
len(usable_patients)

92

In [12]:
usable_patient_df = full_label_df.loc[(full_label_df['patient_id'].isin(usable_patients)) & 
                                      (full_label_df['ground_truth'] != "OTHER")].copy()

In [13]:
usable_patient_df

,ground_truth,image,sd,patient_id,exam_id
39,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,12-COR__T1_POST_GD_FLAIR-32689,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
41,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,11-AX_T1_POST_GD_FLAIR-81300,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
43,T1,/working/lupolab/julia/tcia_analysis/datasets/...,8-AX_T1_pre_gd-14019,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
44,T1,/working/lupolab/julia/tcia_analysis/datasets/...,9-3D_DCE_T1_MAP-73753,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
46,T2,/working/lupolab/julia/tcia_analysis/datasets/...,7-AX_T2_FR-FSE_RF2_150-55295,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
47,T2_FLAIR,/working/lupolab/julia/tcia_analysis/datasets/...,6-AXIAL_FLAIR-84159,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
48,T1,/working/lupolab/julia/tcia_analysis/datasets/...,3-SAG_T1_FLAIR-94337,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
52,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,1001-T1_SE_POST-94212,TCGA-CS-6186,06-01-2000-MRI_BRAIN_per_R-97608
53,T2,/working/lupolab/julia/tcia_analysis/datasets/...,501-T2-WHOLE_BRAIN-32557,TCGA-CS-6186,06-01-2000-MRI_BRAIN_per_R-97608
54,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,1301-T1_SAG_POST-08017,TCGA-CS-6186,06-01-2000-MRI_BRAIN_per_R-97608


In [21]:
commands = []
for idx, row in usable_patient_df.iterrows(): 
    moving_vol = row['image']
    old_vol = moving_vol.split('.n')[0]+'a'+'.nii.gz'
    output_vol = moving_vol.split('.n')[0]+'a_n'+'.nii.gz'
    command_align = 'bet '+old_vol+' '+output_vol+' -S'
    commands.append(command_align)
    

In [22]:
commands

['bet /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/12-COR__T1_POST_GD_FLAIR-32689a.nii.gz /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/12-COR__T1_POST_GD_FLAIR-32689a_n.nii.gz -S',
 'bet /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/11-AX_T1_POST_GD_FLAIR-81300a.nii.gz /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/11-AX_T1_POST_GD_FLAIR-81300a_n.nii.gz -S',
 'bet /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/8-AX_T1_pre_gd-14019a.nii.gz /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/8-AX_T1_pre_gd-14019a_n.nii.gz -S',
 'bet /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/

In [16]:
os.chdir('/working/lupolab/julia/tcia_analysis/code/')

In [24]:
x = open('skullstrip_lgg.txt', 'w')

In [25]:
x.write('{')
x.write('\n')
for command in commands: 
    x.write(command)
    x.write('\n')
x.write('}')
x.close()

In [26]:
!grid_pipeline --scriptonly skullstrip_lgg.txt
